## Imports

In [6]:
from model_fct import ProteinClassifier, ProteinDataModule, ProteinSequenceDataset
import os
import torch
from torch import nn
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torchmetrics
from pytorch_lightning.accelerators import MPSAccelerator
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torchmetrics.classification import MulticlassAUROC, MulticlassAccuracy, MultilabelF1Score
from torchmetrics import Recall, Precision

from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
import datetime
from datetime import datetime
#from pytorch_lightning.metrics.sklearns import Accuracy

import torchvision

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import platform
platform.processor()

'arm'

In [8]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')
val_df = pd.read_pickle('val_df.pkl')
blind_df = pd.read_pickle('blind_df.pkl')

## Logger and checkpoint

In [9]:
def setup_testube_logger() -> CSVLogger:
    """ Function that sets the TestTubeLogger to be used. """
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y--%H-%M-%S")

    return CSVLogger(
        save_dir="experiments/",
        version=dt_string,
        name="lightning_logs",
    )

logger = setup_testube_logger()

In [10]:
ckpt_path = os.path.join(
    logger.save_dir,
    logger.name,
    f"version_{logger.version}",
    "checkpoints",
)

c = ModelCheckpoint(
    dirpath=ckpt_path + "/" + "tanh_3epochs",
    verbose=True,
    monitor='val_acc',
    mode="max",
)

## Set up experiment

In [11]:
TARGETS = ['cyto', 'mito', 'nucleus','other', 'secreted']
PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert_bfd_localization'
#PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert_bfd'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=False)

EPOCHS = 3
BATCH_SIZE = 1
MAX_LENGTH = 1500

In [12]:
dm = ProteinDataModule(
    train_df, 
    test_df,
    val_df,
    blind_df,
    tokenizer, 
    target_list=TARGETS,
    batch_size=BATCH_SIZE,
    max_len = MAX_LENGTH
)

model = ProteinClassifier(
    n_classes=5,
    target_list=TARGETS,
    steps_per_epoch=len(train_df)//BATCH_SIZE, 
    n_epochs=EPOCHS
)

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
trainer = pl.Trainer(max_epochs=EPOCHS,
                     logger=logger,
                     accelerator='mps',
                     #callbacks = checkpoint_callback
                     default_root_dir='experiments/lightning_logs'
                    )

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model, dm)

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name       | Type             | Params
------------------------------------------------
0 | bert       | BertModel        | 419 M 
1 | classifier | Sequential       | 5.1 K 
2 | criterion  | CrossEntropyLoss | 0     
------------------------------------------------
419 M     Trainable params
0         Non-trainable params
419 M     Total params
1,679.745 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


the accuracy is 0.00
the precision is 0.00
the recall is 0.00
the f1 is 0.00
   precision  recall   f1  accuracy  num_samples
0        0.0     0.0  0.0       0.0            1
1        0.0     0.0  0.0       1.0            0
2        0.0     0.0  0.0       0.0            1
[[0 1 0]
 [0 0 0]
 [0 1 0]]


/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/User

...

the accuracy is 0.71
the precision is 0.76
the recall is 0.73
the f1 is 0.74
   precision    recall        f1  accuracy  num_samples
0   0.591265  0.523810  0.555497  0.523810         1008
1   0.800604  0.667506  0.728022  0.667506          397
2   0.629154  0.802505  0.705334  0.802505         1038
3   0.897683  0.734597  0.807993  0.734597          633
4   0.888258  0.905405  0.896750  0.905405          518
[[528  42 385  30  23]
 [ 59 265  59   6   8]
 [163  19 833  14   9]
 [115   2  32 465  19]
 [ 28   3  15   3 469]]


/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Validation: 0it [00:00, ?it/s]

the accuracy is 0.74
the precision is 0.78
the recall is 0.75
the f1 is 0.76
   precision    recall        f1  accuracy  num_samples
0   0.630592  0.578042  0.603175  0.578042         1512
1   0.827292  0.652101  0.729323  0.652101          595
2   0.662434  0.804110  0.726429  0.804110         1557
3   0.875581  0.793467  0.832504  0.793467          949
4   0.895541  0.904762  0.900128  0.904762          777
[[ 874   54  495   57   32]
 [ 100  388   80   17   10]
 [ 246   21 1252   26   12]
 [ 123    2   43  753   28]
 [  43    4   20    7  703]]


`Trainer.fit` stopped: `max_epochs=3` reached.


In [15]:
trainer.test(dataloaders=dm)

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:124: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at experiments/lightning_logs/22-02-2023--10-22-41/checkpoints/epoch=2-step=21549.ckpt
Loaded model weights from checkpoint at experiments/lightning_logs/22-02-2023--10-22-41/checkpoints/epoch=2-step=21549.ckpt


Testing: 0it [00:00, ?it/s]

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


the accuracy is 0.79
the precision is 0.83
the recall is 0.79
the f1 is 0.81
   precision    recall        f1  accuracy  num_samples
0   0.687988  0.727723  0.707298  0.727723          606
1   0.937853  0.631179  0.754545  0.631179          263
2   0.743976  0.769470  0.756508  0.769470          642
3   0.838202  0.903148  0.869464  0.903148          413
4   0.943396  0.934579  0.938967  0.934579          321
[[441   7 118  35   5]
 [ 54 166  26  16   1]
 [125   1 494  18   4]
 [  8   3  21 373   8]
 [ 13   0   5   3 300]]


[{}]

In [ ]:
TARGETS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Define the given matrix
classes = TARGETS
matrix = np.array([[475, 21, 94, 8, 8],
                   [46, 203, 7, 6, 1],
                   [153, 7, 473, 6, 3],
                   [18, 18, 3, 373, 1],
                   [23, 3, 1, 5, 289]])

# Initialize the confusion matrix with zeros
confusion_matrix = np.zeros((len(classes), len(classes)))

# Fill the confusion matrix with values from the given matrix
for i in range(len(classes)):
    for j in range(len(classes)):
        confusion_matrix[i, j] = matrix[i, j]

# Normalize the confusion matrix if needed
#confusion_matrix = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.matshow(confusion_matrix, cmap=plt.cm.Oranges)

# Set labels for x and y axis
ax.set_xticklabels([''] + classes, fontsize=12)
ax.set_yticklabels([''] + classes, fontsize=12)

# Set the title and axis labels
ax.set_title('Confusion Matrix', fontsize=16)
ax.set_xlabel('Predicted Label', fontsize=14)
ax.set_ylabel('True Label', fontsize=14)

# Add colorbar
cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.tick_params(labelsize=12)

# Add text annotations to the confusion matrix
for i in range(len(classes)):
    for j in range(len(classes)):
        ax.text(j, i, int(confusion_matrix[i, j]), ha='center', va='center', color='white', fontsize=12)

# Show the plot
#plt.show()
plt.savefig('../Confusion_Matrix_Testing_Set.png', dpi=500, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support


# example target and output lists
targets = [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
outputs = [0, 1, 1, 3, 4, 0, 2, 2, 3, 4]

def classification_metrics(targets, outputs):
    # compute confusion matrix
    cm = confusion_matrix(targets, outputs)
    
    # compute total number of samples for each class
    total_per_class = np.sum(cm, axis=1)
    
    # compute number of correctly classified samples for each class
    correct_per_class = np.diagonal(cm)
    
    # compute precision, recall, and f1 score for each class
    p, r, f1, _ = precision_recall_fscore_support(targets, outputs, average=None)
    
    # compute accuracy for each class
    accuracy_per_class = np.divide(correct_per_class, total_per_class, where=total_per_class!=0)
    
    # create a dataframe to hold the results
    df = pd.DataFrame({
        'precision': p,
        'recall': r,
        'f1': f1,
        'accuracy': accuracy_per_class,
        'num_samples': total_per_class
    })
    
    print(df)
    print(cm)
    #return df

In [ ]:
classification_metrics(targets, outputs)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Define your target and output lists
targets = [0, 1, 2, 3, 4]
outputs = [1, 1, 2, 3, 4]

# Create a confusion matrix
cm = confusion_matrix(targets, outputs)

# Print the confusion matrix
print("Confusion Matrix:\n", cm)

# Calculate classification report
report = classification_report(targets, outputs)

# Print classification report
print("Classification Report:\n", report)

In [ ]:
df.groupby('target')

## Testing and predicting

In [ ]:
#change for best one - manually check which one is the best
best_checkpoint_path = '/Users/pierredemetz/UCL_work/COMP0082-CW/code/experiments/lightning_logs/20-02-2023--21-58-19/checkpoints/epoch=1-step=14366.ckpt'




In [ ]:
trainer = Trainer(resume_from_checkpoint=best_checkpoint_path)

In [16]:
outputs = trainer.predict(model, dm)
results = []
for item in outputs:
    tensor = item[1]
    max_prob, max_target_idx = torch.max(tensor, dim=1)
    max_target = TARGETS[max_target_idx]
    results.append((max_prob.item(), max_target))

print(results)

Predicting: 7183it [00:00, ?it/s]

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[(0.9997225403785706, 'other'), (0.9989412426948547, 'other'), (0.9995458126068115, 'mito'), (0.9990085959434509, 'cyto'), (0.9990605711936951, 'cyto'), (0.9995036125183105, 'mito'), (0.999674916267395, 'secreted'), (0.9995205402374268, 'mito'), (0.9996706247329712, 'secreted'), (0.951432466506958, 'secreted'), (0.9990999698638916, 'cyto'), (0.9989149570465088, 'cyto'), (0.9992892146110535, 'nucleus'), (0.9989752769470215, 'cyto'), (0.9998076558113098, 'other'), (0.9994365572929382, 'nucleus'), (0.9997408986091614, 'other'), (0.9994614720344543, 'nucleus'), (0.9996789693832397, 'other'), (0.9989711046218872, 'cyto')]


In [17]:
outputs

[(0, tensor([[2.6261e-05, 6.6534e-05, 4.7680e-05, 9.9972e-01, 1.3703e-04]])),
 (0, tensor([[2.8934e-04, 1.0329e-04, 6.1720e-05, 9.9894e-01, 6.0441e-04]])),
 (0, tensor([[1.7493e-04, 9.9955e-01, 5.1117e-05, 1.0878e-04, 1.1941e-04]])),
 (0, tensor([[9.9901e-01, 1.4473e-04, 7.0163e-04, 6.6101e-05, 7.8964e-05]])),
 (0, tensor([[9.9906e-01, 1.9037e-04, 5.0704e-04, 1.0551e-04, 1.3662e-04]])),
 (0, tensor([[2.0273e-04, 9.9950e-01, 5.4851e-05, 1.1594e-04, 1.2289e-04]])),
 (0, tensor([[9.2876e-05, 2.0926e-05, 1.5821e-04, 5.3080e-05, 9.9967e-01]])),
 (0, tensor([[1.7974e-04, 9.9952e-01, 4.1405e-05, 1.1627e-04, 1.4197e-04]])),
 (0, tensor([[8.7799e-05, 2.2022e-05, 1.5982e-04, 5.9769e-05, 9.9967e-01]])),
 (0, tensor([[4.6848e-02, 2.7798e-04, 1.1007e-03, 3.4123e-04, 9.5143e-01]])),
 (0, tensor([[9.9910e-01, 1.5329e-04, 5.8158e-04, 5.1600e-05, 1.1361e-04]])),
 (0, tensor([[9.9891e-01, 1.0675e-04, 8.5326e-04, 5.5866e-05, 6.9128e-05]])),
 (0, tensor([[5.6326e-04, 2.0238e-05, 9.9929e-01, 1.8695e-05, 1.

## LEGACY

In [ ]:
target_list = ['cyto', 'mito', 'nucleus','other', 'secreted']
n_classes = 5

protein_classifier = ProteinClassifier(n_classes, target_list)
protein_classifier = protein_classifier.load_from_checkpoint(
    checkpoint_path=best_checkpoint_path,
    n_classes=n_classes,
    target_list=target_list
)

protein_classifier.eval()
protein_classifier.freeze()

In [ ]:
sample = {
  "seq": "M S T D T G V S L P S Y E E D Q G S K L I R K A K E A P F V P V G I A G F A A I V A Y G L Y K L K S R G N T K M S I H L I H M R V A A Q G F V V G A M T V G M G Y S M Y R E F W A K P K P",
}

predictions = protein_classifier.predict_step(sample, batch_idx=0)

print("Sequence Localization Ground Truth is: {} - prediction is: {}".format('Mitochondrion',predictions['predicted_label']))



## MISC

In [ ]:
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print ("MPS device not found.")

In [ ]:
accelerator_registry=torch.device("mps")

In [ ]:
accelertorch.backends.mps

In [ ]:
MPSAccelerator.register_accelerators(device='mps')

In [ ]:
!pip install tensorflow-metal

In [ ]:
import transformers

In [ ]:
!exit

In [ ]:
!arch